In [20]:
using Plots, KernelDensity, Serialization, MCMCChains, Statistics, StableRNGs
include("ode_problem.jl")
include("posterior_analysis.jl");

### Generate Predictions of Training and Test Data

In [21]:
thinned = "1000"

"1000"

Predictions for active G Protein Timecourse

In [22]:
for approach in ["regularized", "unregularized"]
    #load posterior samples and convert to array
    mcmc_chain = deserialize("outputs/004_posterior_samples_$(approach)_ensemble1_walkers1000_iter30000_thinned$(thinned).jls")
    array = Array(mcmc_chain, append_chains=true) #iterations x parameters
    n_iterations = size(array)[1] #count number of parameter vectors
    #convert parameter values from logscale to linear scale 
    samples_linear_scale = [convert_to_linear_scale_from_logscale(array[i,:]) for i in 1:n_iterations];
    #load ODEProblem inputs and create ODEProblem
    odesys, u0, tspan, p = return_ode_problem_default_inputs()
    op = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

    #timecourse predictions
    #specify data output
    data = "timecourse"
    solver_inputs = return_ode_problem_solver_default_inputs(data);

    #generate timecourse predictions using samples 
    fraction_activated_list = []
    fraction_activated_finegrain_list = []
    for i in 1:n_iterations
        parameters = samples_linear_scale[i]
        u0, p = parameter_mapping(parameters)
        output = active_G_predictions(op, u0, p, solver_inputs)
        push!(fraction_activated_list, output)

        #redefine tspan to be more finegrain, for visualization
        solver_inputs_finegrain = copy(solver_inputs)
        solver_inputs_finegrain["saveat"] = collect(range(start=0.0,stop=600.0, length=100))
        output = active_G_predictions(op, u0, p, solver_inputs_finegrain)
        push!(fraction_activated_finegrain_list, output)
    end
    serialize("outputs/006_predictions_active_G_$(data)_$(approach)_$(n_iterations)_samples.jls", fraction_activated_list)
    serialize("outputs/006_predictions_active_G_$(data)_$(approach)_$(n_iterations)_samples_finegrain.jls", fraction_activated_finegrain_list)
end

Predictions for active G Protein Dose Response

In [3]:
for approach in ["regularized", "unregularized"]
    #load posterior samples and convert to array
    mcmc_chain = deserialize("outputs/004_posterior_samples_$(approach)_ensemble1_walkers1000_iter30000_thinned$(thinned).jls")
    array = Array(mcmc_chain, append_chains=true) #iterations x parameters
    n_iterations = size(array)[1] #count number of parameter vectors
    #convert parameter values from logscale to linear scale 
    samples_linear_scale = [convert_to_linear_scale_from_logscale(array[i,:]) for i in 1:n_iterations];
    #load ODEProblem inputs and create ODEProblem
    odesys, u0, tspan, p = return_ode_problem_default_inputs()
    op = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

    #get solver inputs for dose response
    data = "dose_response"
    solver_inputs = return_ode_problem_solver_default_inputs(data)
    #get dosages for dose response
    experiment_dose_response = deserialize("outputs/000_processed_active_G_dose_response.dict");
    ligand_dosages_molecules = experiment_dose_response["ligand_stimulation (molecules)"]
    normalize_to_dose = experiment_dose_response["normalize_to_response_at_dose"];

    #generate dose response predictions using samples 
    fraction_activated_list = []
    fraction_activated_finegrain_list = []
    for i in 1:n_iterations
        parameters = samples_linear_scale[i]
        
        dose_response_i = []
        for l in ligand_dosages_molecules
            u0, p = parameter_mapping_dose_response(parameters, l)
            output = active_G_predictions(op, u0, p, solver_inputs)
            append!(dose_response_i, output)
        end
        u0, p = parameter_mapping_dose_response(parameters, normalize_to_dose)
        normalize_to = active_G_predictions(op, u0, p, solver_inputs)
        scaled_response = dose_response_i./normalize_to;
        push!(fraction_activated_list, scaled_response)

        #define ligand concentration range to be more finegrain, for visualization
        ligand_dosages_molecules_finegrain = 10.0 .^collect(range(start=-2,stop=3,length=100)) .* 1e-9 .*6.022e23
        dose_response_i = []
        for l in ligand_dosages_molecules_finegrain
            u0, p = parameter_mapping_dose_response(parameters, l)
            output = active_G_predictions(op, u0, p, solver_inputs)
            append!(dose_response_i, output)
        end
        u0, p = parameter_mapping_dose_response(parameters, normalize_to_dose)
        normalize_to = active_G_predictions(op, u0, p, solver_inputs)
        scaled_response = dose_response_i./normalize_to;
        push!(fraction_activated_finegrain_list, scaled_response)
    end

    serialize("outputs/006_predictions_active_G_$(data)_$(approach)_$(n_iterations)_samples.jls", fraction_activated_list)
    serialize("outputs/006_predictions_active_G_$(data)_$(approach)_$(n_iterations)_samples_finegrain.jls", fraction_activated_finegrain_list)
end

Predictions for Binding Affinity

In [4]:
for approach in ["regularized", "unregularized"]
    #load posterior samples and convert to array
    mcmc_chain = deserialize("outputs/004_posterior_samples_$(approach)_ensemble1_walkers1000_iter30000_thinned$(thinned).jls")
    array = Array(mcmc_chain, append_chains=true) #iterations x parameters
    n_iterations = size(array)[1] #count number of parameter vectors
    #convert parameter values from logscale to linear scale 
    samples_linear_scale = [convert_to_linear_scale_from_logscale(array[i,:]) for i in 1:n_iterations];
    #load ODEProblem inputs and create ODEProblem
    odesys, u0, tspan, p = return_ode_problem_default_inputs()
    op = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

    #get solver inputs for dose response
    data = "dose_response"
    solver_inputs = return_ode_problem_solver_default_inputs(data)
    #get dosages for dose response
    experiment_dose_response = deserialize("outputs/000_processed_active_G_dose_response.dict");
    ligand_dosages_molecules = experiment_dose_response["ligand_stimulation (molecules)"]
    normalize_to_dose = experiment_dose_response["normalize_to_response_at_dose"];

    #generate dose response predictions using samples 
    fraction_activated_list = []
    fraction_activated_finegrain_list = []
    for i in 1:n_iterations
        parameters = samples_linear_scale[i]
        
        dose_response_i = []
        for l in ligand_dosages_molecules
            u0, p = parameter_mapping_dose_response(parameters, l)
            output = RL_predictions(op, u0, p, solver_inputs)
            append!(dose_response_i, output)
        end
        u0, p = parameter_mapping_dose_response(parameters, normalize_to_dose)
        normalize_to = RL_predictions(op, u0, p, solver_inputs)
        scaled_response = dose_response_i./normalize_to;
        push!(fraction_activated_list, scaled_response)

        #redefine ligand concentration range to be more finegrain, for visualization
        ligand_dosages_molecules_finegrain = 10.0 .^collect(range(start=-2,stop=3,length=100)) .* 1e-9 .*6.022e23
        dose_response_i = []
        for l in ligand_dosages_molecules_finegrain
            u0, p = parameter_mapping_dose_response(parameters, l)
            output = RL_predictions(op, u0, p, solver_inputs)
            append!(dose_response_i, output)
        end
        u0, p = parameter_mapping_dose_response(parameters, normalize_to_dose)
        normalize_to = RL_predictions(op, u0, p, solver_inputs)
        scaled_response = dose_response_i./normalize_to;
        push!(fraction_activated_finegrain_list, scaled_response)
    end

    serialize("outputs/006_predictions_binding_affinity_$(data)_$(approach)_$(n_iterations)_samples.jls", fraction_activated_list)
    serialize("outputs/006_predictions_binding_affinity_$(data)_$(approach)_$(n_iterations)_samples_finegrain.jls", fraction_activated_finegrain_list)
end

LoadError: InterruptException:

In [5]:
for approach in ["regularized", "unregularized"]
    #load posterior samples and convert to array
    mcmc_chain = deserialize("outputs/004_posterior_samples_$(approach)_ensemble1_walkers1000_iter30000_thinned$(thinned).jls")
    array = Array(mcmc_chain, append_chains=true) #iterations x parameters
    n_iterations = size(array)[1] #count number of parameter vectors
    #convert parameter values from logscale to linear scale 
    samples_linear_scale = [convert_to_linear_scale_from_logscale(array[i,:]) for i in 1:n_iterations];
    #load ODEProblem inputs and create ODEProblem
    odesys, u0, tspan, p = return_ode_problem_default_inputs()
    op = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

    #get solver inputs for dose response
    data = "dose_response"
    solver_inputs = return_ode_problem_solver_default_inputs(data)
    #get dosages for dose response
    experiment_dose_response = deserialize("outputs/000_processed_active_G_dose_response.dict");
    ligand_dosages_molecules = experiment_dose_response["ligand_stimulation (molecules)"]
    normalize_to_dose = experiment_dose_response["normalize_to_response_at_dose"];

    #generate dose response predictions using samples 
    fraction_activated_list = []
    for i in 1:n_iterations
        parameters = samples_linear_scale[i]
        
        dose_response_i = []
        for l in ligand_dosages_molecules
            u0, p = parameter_mapping_dose_response(parameters, l)
            output = RL_predictions(op, u0, p, solver_inputs)
            append!(dose_response_i, output)
        end
        u0, p = parameter_mapping_dose_response(parameters, normalize_to_dose)
        normalize_to = RL_predictions(op, u0, p, solver_inputs)
        scaled_response = dose_response_i./normalize_to;
        push!(fraction_activated_list, scaled_response)

    end
    serialize("outputs/006_predictions_binding_affinity_$(data)_$(approach)_$(n_iterations)_samples.jls", fraction_activated_list)
end